In [1]:
import os,sys
b_directory = os.path.abspath(os.path.join(os.getcwd(),'..', '..', 'model'))
sys.path.insert(0, b_directory)
from specific_epitope import *
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.optim import Adam
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from timm.models.vision_transformer import Mlp
from sklearn.metrics import (
    average_precision_score,
    roc_auc_score
)

/home/taoyuhao/anaconda3/envs/sx/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
class sx_Dataset(Dataset):
    def __init__(self,data1,data2,data3,data4,data5):
        self.x1 = data1
        self.x2 = data2
        self.x3 = data3
        self.x4 = data4
        self.x5 = data5

        self.len = data1.shape[0]
        
    def __getitem__(self, index):
        return self.x1[index],self.x2[index],self.x3[index],self.x4[index],self.x5[index]
 
    def __len__(self):
        return self.len

In [3]:
def cal_(beta_train_emb,beta_train_st,alpha_train_emb,alpha_train_st,
         train_labels,seed=1,lr=0.0001,weight_decay=0.02,
         num_blocks=1,latent_dim=256,BATCH_SIZE=4,epoch=200,device='cuda:0',save_path='./model.pt'):
    torch.manual_seed(seed)
    dataset=sx_Dataset(beta_train_emb,beta_train_st,
                       alpha_train_emb,alpha_train_st,train_labels)
    model=classification_model(beta_input_dim=beta_train_emb.shape[-1],beta_mid_dim=beta_train_emb.shape[-2],
                                beta_st_input_dim=beta_train_st.shape[-1],beta_st_mid_dim=beta_train_st.shape[-2],
                                alpha_input_dim=alpha_train_emb.shape[-1],alpha_mid_dim=alpha_train_emb.shape[-2],
                                alpha_st_input_dim=alpha_train_st.shape[-1],alpha_st_mid_dim=alpha_train_st.shape[-2],
                               num_blocks=num_blocks,latent_dim=latent_dim
                              )
    
    model=model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    total_itm=0
    for es in range(epoch):
        model.train()
        tcr_dataloader = DataLoader(dataset=dataset,batch_size=BATCH_SIZE,shuffle=True,num_workers=4,drop_last=True)
  
        for i,(be,bt,ae,at,labels) in enumerate(tcr_dataloader):
            be=torch.tensor(be,dtype=torch.float32)
            be=be.to(device)
            bt=torch.tensor(bt,dtype=torch.float32)
            bt=bt.to(device)
            
            ae=torch.tensor(ae,dtype=torch.float32)
            ae=be.to(device)
            at=torch.tensor(at,dtype=torch.float32)
            at=at.to(device)
            
            itm_labels=torch.tensor(labels,dtype=torch.float32)
            itm_labels=itm_labels.to(device)

            vl_output = model(be,bt,ae,at).flatten()
            loss_itm = F.binary_cross_entropy(vl_output.squeeze(), itm_labels, reduction='none').mean()

            total_itm+=loss_itm.item()

            optimizer.zero_grad()
            loss_itm.backward()
            optimizer.step()
            
    torch.save(model,save_path)

In [4]:
beta_train_emb=np.load('../../tmp_data/1/beta_train_emb.npy')
beta_train_st=np.load('../../tmp_data/1/beta_train_st.npy')
alpha_train_emb=np.load('../../tmp_data/1/alpha_train_emb.npy')
alpha_train_st=np.load('../../tmp_data/1/alpha_train_st.npy')
train_labels=pd.read_csv('../../tmp_data/1/train.csv')['Label'].to_numpy()
model_save_path='./model.pt'
cuda='cuda:0'

In [5]:
cal_(beta_train_emb,beta_train_st,alpha_train_emb,alpha_train_st,
         train_labels,device=cuda,save_path=model_save_path)

/tmp/ipykernel_3881236/1359823471.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  be=torch.tensor(be,dtype=torch.float32)
/tmp/ipykernel_3881236/1359823471.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bt=torch.tensor(bt,dtype=torch.float32)
/tmp/ipykernel_3881236/1359823471.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ae=torch.tensor(ae,dtype=torch.float32)
/tmp/ipykernel_3881236/1359823471.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone

In [6]:
model=torch.load(model_save_path)

/tmp/ipykernel_3881236/3839462922.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model=torch.load(model_save_path)


In [7]:
model.eval()

classification_model(
  (exp_linear): Linear(in_features=64, out_features=256, bias=True)
  (low_linear): Linear(in_features=1024, out_features=256, bias=True)
  (blks): ModuleList(
    (0): AttentionBlock(
      (norm1): BatchNorm1d(121, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True)
      (norm2): BatchNorm1d(121, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True)
      (attn): Attention(
        (qkv): Linear(in_features=256, out_features=768, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=256, out_features=256, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (mlp): Mlp(
        (fc1): Linear(in_features=256, out_features=1024, bias=True)
        (act): GELU(approximate='tanh')
        (drop1): Dropout(p=0, inplace=False)
        (fc2): Linear(in_features=1024, out_features=256, bias=True)
        (drop2): Dropout(p=0, inplace=False)
      )
    )
  )
  (dense): Sequential(
  

In [8]:
beta_test_emb=np.load('../../tmp_data/1/beta_test_emb.npy')
beta_test_st=np.load('../../tmp_data/1/beta_test_st.npy')
alpha_test_emb=np.load('../../tmp_data/1/alpha_test_emb.npy')
alpha_test_st=np.load('../../tmp_data/1/alpha_test_st.npy')
test_labels=pd.read_csv('../../tmp_data/1/test.csv')['Label'].to_numpy()

In [9]:
preds=model(torch.tensor(beta_test_emb,dtype=torch.float32).to(cuda),
            torch.tensor(beta_test_st,dtype=torch.float32).to(cuda),
            torch.tensor(alpha_test_emb,dtype=torch.float32).to(cuda),
            torch.tensor(alpha_test_st,dtype=torch.float32).to(cuda))

In [10]:
roc_auc_score(test_labels,preds.detach().cpu().numpy())

1.0